# 区域卷积神经网络（R-CNN）

* 区域卷积神经网络（region-based CNN或regions with CNN features，R-CNN） 
* 快速的R-CNN（Fast R-CNN）
* 更快的R-CNN（Faster R-CNN）
* 掩码R-CNN（Mask R-CNN）

## R-CNN

* 阅读：区域卷积神经网络\_R-CNNs.pdf
* 每个提议区域都将被标注类别和真实边界框
* 将每个提议区域变形为网络需要的输入尺寸


* 训练多个支持向量机对目标分类
* 训练线性回归模型来预测真实边界框

### 不足

* 速度很慢
* 可能从一张图像中选出上千个提议区域
* 上千次的卷积神经网络的前向传播

## Fast R-CNN

**回顾R-CNN**

* 对每个提议区域，卷积神经网络的前向传播是独立的，而没有共享计算
* 这些区域通常有重叠，独立的特征抽取会导致重复的计算

### region of interest (RoI) pooling layer

* 仅在整张图象上执行卷积神经网络的前向传播
* Suppose that selective search generates $n$ region proposals
* These region proposals (of different shapes) mark regions of interest (of different shapes) on the CNN output

* 输入图像：$40\times 40$
* 
```
X = torch.arange(16.).reshape(1, 1, 4, 4)
tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])
```
* 两个提议区域
```
rois = torch.Tensor([[0, 0, 0, 20, 20], [0, 0, 10, 30, 30]])
```

* `spatial_scale=0.1`
* 两个兴趣区域`X[:, :, 0:3, 0:3]`和`X[:, :, 1:4, 0:4]`
* 这些感兴趣的区域需要进一步抽取出形状相同的特征（in order to be easily concatenated）

### RoI pooling layer

* 每个区域输出的高和宽分别为$h_2$和$w_2$
* 形状为$h \times w$的兴趣区域窗口
* 该窗口将被划分为$h_2 \times w_2$子窗口网格，其中每个子窗口的大小约为$\tfrac{h}{h_2} \times \tfrac{w}{w_2}$
* 最大元素作为该子窗口的输出
* 阅读：区域卷积神经网络\_R-CNNs.pdf

阅读实例：

```
torchvision.ops.roi_pool(X, 
    rois, output_size=(2, 2), spatial_scale=0.1)
```

## Faster R-CNN

阅读：区域卷积神经网络\_R-CNNs.pdf

* transform the CNN output to a new output with $c$ channels：$3\times 3$卷积，填充1

### region proposal network

* 以特征图的每个像素为中心，generate multiple anchor boxes of different scales and aspect ratios and label them
* $c$ channels：分别预测锚框的二元类别（含目标还是背景）和边界框
* NMS：从预测类别为目标的预测边界框中移除相似的结果

Faster R-CNN的目标函数中：区域提议网络中锚框的二元类别和边界框预测